# Load Senitnel-2 data from the CDSE STAC catalog

This notebook shows an example how to load Sentinel-2 data from the [CDSE STAC API](https://documentation.dataspace.copernicus.eu/APIs/STAC.html).

### Setup
In order to run this notebook you may install [`xcube_stac`](https://github.com/xcube-dev/xcube-stac). You may install [`xcube_stac`](https://github.com/xcube-dev/xcube-stac) directly from the git repository by cloning the repository, directing into `xcube-stac`, and following the steps below:

```bash
conda env create -f environment.yml
conda activate xcube-stac
pip install .
```

Note that [`xcube_stac`](https://github.com/xcube-dev/xcube-stac) is a plugin of [`xcube`](https://xcube.readthedocs.io/en/latest/), where `xcube` is included in the `environment.yml`.  

The data can be accessed via S3, where key and secret can be obtained following the [CDSE access documentation to EO data via S3](https://documentation.dataspace.copernicus.eu/APIs/S3.html#generate-secrets). The store object will receive the key and secret upon initialization, as demonstrated below.

Now, we first import everything we need:

In [1]:
%%time
from xcube.core.store import new_data_store, get_data_store_params_schema
import itertools

CPU times: user 2.85 s, sys: 189 ms, total: 3.04 s
Wall time: 1.19 s


Next store the credentials in a dictionary. 

In [2]:
credentials = {
    "key": "O0M0CUQIDQO9TDZ4D8NR",
    "secret": "qPUyXs9G6j8on6MY5KPhQNHuA5uZTqxEscrbBCGx"
}

First, we get the store parameters needed to initialize a STAC [data store](https://xcube.readthedocs.io/en/latest/dataaccess.html#data-store-framework). Note that key and secret of the S3 access are required.

In [3]:
%%time
store_params = get_data_store_params_schema("stac-cdse")
store_params

CPU times: user 620 ms, sys: 55.9 ms, total: 676 ms
Wall time: 676 ms


Note that the user does not need to provide the URL for the [CDSE STAC API](https://documentation.dataspace.copernicus.eu/APIs/STAC.html). Only the key and secret for S3 access are required when initializing a `stac-cdse` [data store](https://xcube.readthedocs.io/en/latest/dataaccess.html#data-store-framework). First, we will initialize a store supporting the stacking mode. Then, for completeness, we will initialize a store in single-tile mode.

In [4]:
%%time
store = new_data_store("stac-cdse", stack_mode=True, **credentials)

CPU times: user 11.5 ms, sys: 3.7 ms, total: 15.2 ms
Wall time: 167 ms


The data IDs point to a [STAC item's JSON](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md) and are specified by the segment of the URL that follows the catalog's URL. The data IDs can be streamed using the following code where we show the first 10 data IDs as an example.

In [5]:
%%time
data_ids = store.list_data_ids()
data_ids

CPU times: user 20.5 ms, sys: 439 μs, total: 20.9 ms
Wall time: 75 ms


['COP-DEM',
 'S2GLC',
 'TERRAAQUA',
 'SENTINEL-3',
 'SENTINEL-5P',
 'SENTINEL-1-RTC',
 'SENTINEL-1',
 'SMOS',
 'LANDSAT-7',
 'CCM',
 'LANDSAT-5',
 'LANDSAT-8',
 'ENVISAT',
 'SENTINEL-6',
 'GLOBAL-MOSAICS',
 'SENTINEL-2']

Below, the parameters for the `open_data` method can be viewed. Note that `processing_level` and and [`processing_baseline`](https://sentiwiki.copernicus.eu/web/s2-processing#S2Processing-ProcessingBaseline) can be selected. 

In [6]:
%%time
open_params = store.get_open_data_params_schema()
open_params

CPU times: user 31 μs, sys: 3 μs, total: 34 μs
Wall time: 36.2 μs


So far, only data from the collection `SENTINEL-2` can be accessed. We therefore assign `data_id` to `"SENTINEL-2"`. We set the bounding box to cover the greater Hamburg area and the time range to July 2020. Additionally, we assign the processing level to `"L2A"` and the processing baseline to `"5.00"`.

We can apply a reprojection by setting `crs` to the desired output projection. The odc-stac library uses [`rasterio.wrap.reproject`](https://rasterio.readthedocs.io/en/stable/topics/reproject.html) to handle the reprojection. All arguments of [`odc.stac.load`](https://odc-stac.readthedocs.io/en/latest/_api/odc.stac.load.html) can be passed into the `open_data(...)` method, which forwards them to the `odc.stac.load` function.

In [7]:
%%time
ds = store.open_data(
    data_id="SENTINEL-2",
    bbox=[9.4, 53.3, 10.4, 54],
    time_range=["2020-07-01", "2020-08-01"],
    processing_level="L2A",
    processing_baseline="5.00",
    resolution=0.00018,
    crs="EPSG:4326",
    chunks={"time": 1, "x": 2048, "y": 2048},
)
ds

CPU times: user 24.5 s, sys: 522 ms, total: 25.1 s
Wall time: 4min 28s


<xarray.Dataset> Size: 51GB
Dimensions:      (latitude: 5000, longitude: 8890, time: 19)
Coordinates:
  * latitude     (latitude) float64 40kB 54.0 54.0 54.0 54.0 ... 53.1 53.1 53.1
  * longitude    (longitude) float64 71kB 9.1 9.1 9.1 9.101 ... 10.7 10.7 10.7
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 152B 2020-07-01T10:36:29.024000 ... 20...
Data variables: (12/15)
    AOT          (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B01          (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B02          (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B03          (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B04          (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B05          (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    ...           ...
    B8A          (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B09          (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B11          (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    B12          (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    SCL          (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    WVP          (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>

Note that the search function in the CDSE STAC API is very slow. Further investigation and comparison with other STAC APIs is needed. 

Since the tiles of the Sentinel-2 L2A in this STAC catalog are stored as `.jp2`, we can also open the data as a [xcube's multi-resolution dataset](https://xcube.readthedocs.io/en/latest/mldatasets.html#xcube-multi-resolution-datasets), where each level is linked to an overview level of the data pyramid. The number of levels can be extracted as shown below. 

In [8]:
%%time
mlds = store.open_data(
    data_id="SENTINEL-2",
    data_type="mldataset",
    bbox=[9.1, 53.1, 10.7, 54],
    time_range=["2020-07-01", "2020-08-01"],
    processing_level="L2A",
    processing_baseline="5.00",
    resolution=0.00018,
    crs="EPSG:4326",
    chunks={"time": 1, "x": 2048, "y": 2048},
)
mlds.num_levels

The keyword 'resolution' is not considered when opening the data as multi-resolution dataset.


CPU times: user 22.2 s, sys: 441 ms, total: 22.6 s
Wall time: 3min 59s


5

To get a specific level, we use the `get_dataset(idx_level)` method.

In [9]:
%%time
ds = mlds.get_dataset(4)
ds

CPU times: user 23 μs, sys: 1e+03 ns, total: 24 μs
Wall time: 25.5 μs


<xarray.Dataset> Size: 796MB
Dimensions:      (latitude: 629, longitude: 1110, time: 19)
Coordinates:
  * latitude     (latitude) float64 5kB 53.1 53.1 53.1 53.1 ... 54.0 54.0 54.0
  * longitude    (longitude) float64 9kB 9.099 9.101 9.102 ... 10.7 10.7 10.7
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 152B 2020-07-01T10:36:29.024000 ... 20...
Data variables: (12/15)
    AOT          (time, latitude, longitude) float32 53MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B01          (time, latitude, longitude) float32 53MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B02          (time, latitude, longitude) float32 53MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B03          (time, latitude, longitude) float32 53MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B04          (time, latitude, longitude) float32 53MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B05          (time, latitude, longitude) float32 53MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    ...           ...
    B8A          (time, latitude, longitude) float32 53MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B09          (time, latitude, longitude) float32 53MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B11          (time, latitude, longitude) float32 53MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B12          (time, latitude, longitude) float32 53MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    SCL          (time, latitude, longitude) float32 53MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    WVP          (time, latitude, longitude) float32 53MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>

We can plot a tile of the B04 (red) band for a given timestamp as an example. Note that the bounding box encompasses all tiles within the dataset, so without mosaicking, many of the pixels will be NaN values.

In [10]:
%%time
ds.B04.isel(time=-6).plot(vmin=0, vmax=2000)

SystemError: <built-in function exc_info> returned a result with an exception set

To apply mosaicking, we need to assign `groupby="solar_day"`, as shown in the [documentation of `odc.stac.load`](https://odc-stac.readthedocs.io/en/latest/_api/odc.stac.load.html).

In [11]:
%%time
mlds = store.open_data(
    data_id="SENTINEL-2",
    data_type="mldataset",
    bbox=[9.1, 53.1, 10.7, 54],
    time_range=["2020-07-01", "2020-08-01"],
    processing_level="L2A",
    processing_baseline="5.00",
    resolution=0.00018,
    crs="EPSG:4326",
    chunks={"time": 1, "x": 2048, "y": 2048},
    groupby="solar_day",
)

The keyword 'resolution' is not considered when opening the data as multi-resolution dataset.


CPU times: user 23.9 s, sys: 418 ms, total: 24.4 s
Wall time: 3min 34s


We again select the desired level and plot the same example as before. 

In [12]:
%%time
ds = mlds.get_dataset(4)
ds

CPU times: user 30 μs, sys: 1 μs, total: 31 μs
Wall time: 35.5 μs


<xarray.Dataset> Size: 754MB
Dimensions:      (latitude: 629, longitude: 1110, time: 18)
Coordinates:
  * latitude     (latitude) float64 5kB 53.1 53.1 53.1 53.1 ... 54.0 54.0 54.0
  * longitude    (longitude) float64 9kB 9.099 9.101 9.102 ... 10.7 10.7 10.7
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 144B 2020-07-03T10:30:31.024000 ... 20...
Data variables: (12/15)
    AOT          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B01          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B02          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B03          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B04          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B05          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    ...           ...
    B8A          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B09          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B11          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B12          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    SCL          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    WVP          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>

In [13]:
%%time
ds.red.isel(time=-1).plot(vmin=0, vmax=2000)

AttributeError: 'Dataset' object has no attribute 'red'

----
## Data store in the single-tile
For completeness, we initiate the data store in the single-tile mode and open data of one tile. 

In [14]:
%%time
store = new_data_store("stac-cdse", stack_mode=False, **credentials)

CPU times: user 16.7 ms, sys: 4.18 ms, total: 20.9 ms
Wall time: 113 ms


The data IDs point to a [STAC item's JSON](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md) and are specified by the segment of the URL that follows the catalog's URL. The data IDs can be streamed using the following code where we show the first 10 data IDs as an example.

In [15]:
%%time
data_ids = store.get_data_ids()
list(itertools.islice(data_ids, 10))

CPU times: user 6.83 ms, sys: 3.79 ms, total: 10.6 ms
Wall time: 205 ms


['collections/SENTINEL-3/items/S3A_OPER_AUX_GNSSRD_POD__20171212T193419_V20160225T072708_20160225T235942',
 'collections/SENTINEL-3/items/S3A_OPER_AUX_GNSSRD_POD__20171212T194404_V20160226T235943_20160227T235942',
 'collections/SENTINEL-3/items/S3A_OPER_AUX_GNSSRD_POD__20171212T193142_V20160223T235943_20160224T225600',
 'collections/SENTINEL-3/items/S3A_OPER_AUX_GNSSRD_POD__20171212T194851_V20160227T235943_20160228T235942',
 'collections/SENTINEL-3/items/S3A_OPER_AUX_GNSSRD_POD__20171212T212947_V20160317T235943_20160318T235942',
 'collections/SENTINEL-3/items/S3A_OPER_AUX_GNSSRD_POD__20171212T220217_V20160324T235943_20160325T235942',
 'collections/SENTINEL-3/items/S3A_OPER_AUX_GNSSRD_POD__20171212T195832_V20160229T235943_20160301T235942',
 'collections/SENTINEL-3/items/S3A_OPER_AUX_GNSSRD_POD__20171212T214844_V20160321T235943_20160322T235942',
 'collections/SENTINEL-3/items/S3A_OPER_AUX_GNSSRD_POD__20171212T205619_V20160311T235943_20160312T235942',
 'collections/SENTINEL-3/items/S3A_OP

In the next step, we can search for items using search parameters. The following code shows which search parameters are available.

In [16]:
%%time
search_params = store.get_search_params_schema()
search_params

CPU times: user 84 μs, sys: 3 μs, total: 87 μs
Wall time: 94.4 μs


 Note that since only the data access for Sentinel-2 data is supported so far, we will search for tiles of Sentinel-2 data.

In [17]:
%%time
descriptors = list(
    store.search_data(
        collections=["SENTINEL-2"],
        bbox=[9, 47, 10, 48],
        time_range=["2020-07-01", "2020-07-05"],
        processing_level="L1C",
        processing_baseline="2.09",   
    )
)
[d.to_dict() for d in descriptors]

TypeError: Client.search() got an unexpected keyword argument 'processing_level'

In the next step, we can open the data for each data ID. The following code shows which parameters are available for opening the data.

In [18]:
%%time
open_params = store.get_open_data_params_schema()
open_params

CPU times: user 330 μs, sys: 11 μs, total: 341 μs
Wall time: 355 μs


We select the band B04 (red), B03 (green), and B02 (blue) and lazily load the corresponding data.

In [19]:
%%time
ds = store.open_data(
    descriptors[3].data_id,
    asset_names=["B04", "B03", "B02"]
)
ds

NameError: name 'descriptors' is not defined

We plot the loaded data as an example below.

In [20]:
%%time
ds.B04[:5000:10, :5000:10].plot(vmin=0.0, vmax=2000)

SystemError: <built-in function exc_info> returned a result with an exception set

We can also open a `.jp2` as a [xcube's multi-resolution  dataset](https://xcube.readthedocs.io/en/latest/mldatasets.html#xcube-multi-resolution-datasets), where we can select the level of resolution, shown below.  

In [21]:
%%time
mlds = store.open_data(
    descriptors[3].data_id,
    asset_names=["B04", "B03", "B02"],
    data_type="mldataset"
)
mlds.num_levels

NameError: name 'descriptors' is not defined

In [22]:
ds = mlds.get_dataset(4)
ds

<xarray.Dataset> Size: 754MB
Dimensions:      (latitude: 629, longitude: 1110, time: 18)
Coordinates:
  * latitude     (latitude) float64 5kB 53.1 53.1 53.1 53.1 ... 54.0 54.0 54.0
  * longitude    (longitude) float64 9kB 9.099 9.101 9.102 ... 10.7 10.7 10.7
    spatial_ref  int32 4B 4326
  * time         (time) datetime64[ns] 144B 2020-07-03T10:30:31.024000 ... 20...
Data variables: (12/15)
    AOT          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B01          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B02          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B03          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B04          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B05          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    ...           ...
    B8A          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B09          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B11          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    B12          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    SCL          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>
    WVP          (time, latitude, longitude) float32 50MB dask.array<chunksize=(1, 629, 1110), meta=np.ndarray>

In [ ]:
%%time
ds.B04[:312, :312].plot(vmin=0.0, vmax=0.2)